In [1]:
import polars as pl
import geopandas as gp

import ingest

In [2]:
pp = ingest.load_pp()
postcodes = ingest.load_postcodes()

In [3]:
# Get properties joined to lat/longs

property_locations = (
    pp
    .join(
        postcodes,
        on="postcode",
        how="left",
    )
    .with_columns(
        pl.col("transfer_date").dt.year().alias("transfer_year")
    )
    .select(
        "price",
        "transfer_year",
        "street_number",
        "street",
        "postcode",
        "latitude",
        "longitude"
    )
    .collect()
)

property_locations_pandas = property_locations.to_pandas()


In [4]:
# Convert into geodataframe
pl_geo = gp.GeoDataFrame(
    property_locations_pandas,
    geometry=gp.points_from_xy(property_locations_pandas.longitude, property_locations_pandas.latitude,
    # Denotes geodetic degree values
    crs="EPSG:4326"
                               )
)

In [5]:
# Get birmingham ward areas geodataframe
bham_wards = ingest.load_bham_wards()

In [14]:
# Inner join to birmingham ward areas
print("Coords match" if bham_wards.crs == pl_geo.crs else "Coords do NOT match")
pl_geo_joined = pl_geo.sjoin(bham_wards, how="inner")
pl_geo_joined

Coords match


,price,transfer_year,street_number,street,postcode,latitude,longitude,geometry,index_right,geo_point_2d,wd21cd,wd21nm,bng_e,bng_n,lat,long
26,89000,1995,28,SLADE ROAD,B75 5PG,52.588561,-1.810152,POINT (-1.81015 52.58856),17,"{ ""lon"": -1.8010702419042348, ""lat"": 52.583441...",E05011177,Sutton Roughley,413797,298358,52.5829,-1.79780
154,48950,1995,444,BRAYS ROAD,B26 2RT,52.463906,-1.778233,POINT (-1.77823 52.46391),65,"{ ""lon"": -1.7785899167923731, ""lat"": 52.459711...",E05011166,Sheldon,414863,284868,52.4616,-1.78266
184,59950,1995,55,MINIVA DRIVE,B76 2WT,52.548510,-1.793203,POINT (-1.7932 52.54851),19,"{ ""lon"": -1.7754110305569228, ""lat"": 52.537579...",E05011180,Sutton Walmley & Minworth,415562,293480,52.5390,-1.77198
295,45000,1995,54,SANDGATE ROAD,B28 0UL,52.415657,-1.840211,POINT (-1.84021 52.41566),56,"{ ""lon"": -1.8471410433362603, ""lat"": 52.419739...",E05011141,Hall Green South,410572,279991,52.4178,-1.84597
375,50000,1995,12,HANBURY CROFT,B27 6RX,52.447535,-1.809110,POINT (-1.80911 52.44754),66,"{ ""lon"": -1.8128180006560384, ""lat"": 52.458861...",E05011169,South Yardley,412882,284449,52.4579,-1.81184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28270452,186000,2023,101,LILY ROAD,B26 1TE,52.463227,-1.811824,POINT (-1.81182 52.46323),66,"{ ""lon"": -1.8128180006560384, ""lat"": 52.458861...",E05011169,South Yardley,412882,284449,52.4579,-1.81184
28270454,225000,2023,106,GORDON ROAD,B17 9EY,52.461009,-1.948223,POINT (-1.94822 52.46101),57,"{ ""lon"": -1.9605729791830928, ""lat"": 52.457479...",E05011144,Harborne,402791,284633,52.4596,-1.96034
28270455,161500,2023,26,SOMERCOTES ROAD,B42 2JR,52.539983,-1.901171,POINT (-1.90117 52.53998),34,"{ ""lon"": -1.8981309555502432, ""lat"": 52.547667...",E05011159,Oscott,406859,294097,52.5447,-1.90028
28270463,161500,2023,26,SOMERCOTES ROAD,B42 2JR,52.539983,-1.901171,POINT (-1.90117 52.53998),34,"{ ""lon"": -1.8981309555502432, ""lat"": 52.547667...",E05011159,Oscott,406859,294097,52.5447,-1.90028


In [ ]:
# Transform the dataframe back to polars
# to_wkt() transforms the geometry fields into strings
pl_polars = pl.from_pandas(pl_geo_joined.to_wkt())

In [17]:
# Perform aggregations
(
    pl_polars
    .group_by("transfer_year", "wd21nm")
    .agg(pl.median("price").alias("median_price"))
    .sort("transfer_year", "wd21nm", descending=[True, False])
)


transfer_year,wd21nm,median_price
i32,str,f64
2023,"""Acocks Green""",240000.0
2023,"""Allens Cross""",183750.0
2023,"""Alum Rock""",132750.0
2023,"""Aston""",160000.0
2023,"""Balsall Heath West""",100000.0
…,…,…
1995,"""Tyseley & Hay Mills""",37000.0
1995,"""Ward End""",35250.0
1995,"""Weoley & Selly Oak""",45725.0
